In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent.parent
sys.path.append(str(BASE_DIR))

In [3]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3
import numpy as np
import pandas as pd

In [ ]:
DIR = BASE_DIR/'scripts_res'/'Algorithms'
DIR

WindowsPath('d:/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res')

### ASBO

In [5]:
re_run_ASBO = False
re_run_save_ASBO = False
sol = 'ASBO'
case_name = 'Case3'
runs_number = 100

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []
all_res_list = []

def func(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    return -obj,cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

if re_run_ASBO == True or re_run_save_ASBO == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        try:
            optimizer.maximize(init_points=100, n_iter=200,p_hyper=[[5, 2, 0.78, 0.95,1e-5]],acquisition_function=acquisition_function)
            print(optimizer.max)
            x_1_list.extend([optimizer.max['params']['x1']])
            x_2_list.extend([optimizer.max['params']['x2']])
            x_3_list.extend([optimizer.max['params']['x3']])
            x_4_list.extend([optimizer.max['params']['x4']])
            x_5_list.extend([optimizer.max['params']['x5']])  
            obj_list.extend([optimizer.max['target']])                                  
            con_1_list.extend([optimizer.max['constraint'][0]])
            con_2_list.extend([optimizer.max['constraint'][1]])
            con_3_list.extend([optimizer.max['constraint'][2]])
            con_4_list.extend([optimizer.max['constraint'][3]])
            all_res = [item['target'] for item in optimizer.res]
            all_fea = [item['allowed'] for item in optimizer.res]
            modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
            all_res_list.append(modified_res)            
        except Exception as e:
            print(e)
            obj_list.extend(None)
            x_1_list.extend(None)
            x_2_list.extend(None)
            x_3_list.extend(None)
            x_4_list.extend(None)
            x_5_list.extend(None)                        
            con_1_list.extend(None)
            con_2_list.extend(None)
            con_3_list.extend(None)
            con_4_list.extend(None) 
            modified_res = [-np.inf for _ in range(100+200)]
            all_res_list.append(modified_res)   

        df2 = pd.DataFrame(all_res_list, columns=[list(range(100+200))])               
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list
        })
        if re_run_save_ASBO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
            df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)


{'target': -1.0700514641831416, 'params': {'x1': 0.00017339079330783096, 'x2': 6.856984649474057, 'x3': -6.658602590999367, 'x4': -2.678409074393186, 'x5': 2.3815197388790708}, 'constraint': array([-0.67155458,  1.43943719,  1.00421917,  0.35092681])}
{'target': -5.923477228043907, 'params': {'x1': 1.9303229761728156, 'x2': -2.474660576892117, 'x3': -1.225333799922562, 'x4': -2.520640949184257, 'x5': 3.192691637944728}, 'constraint': array([ 1.79824184, -0.50068158,  0.26705189,  0.5805066 ])}
{'target': -1.0761842636274825, 'params': {'x1': 0.002311474454392217, 'x2': -2.308768384495117, 'x3': -5.690283418712296, 'x4': -1.431963390391152, 'x5': 7.365533635010451}, 'constraint': array([ 1.29405392, -0.19611907,  1.13434052,  1.03635967])}
{'target': -1.0700467319593376, 'params': {'x1': -0.00017172097415023952, 'x2': -1.6604481723664537, 'x3': 0.3555589597771337, 'x4': 4.270923931667165, 'x5': 5.667883368436525}, 'constraint': array([1.06870881, 0.94199351, 1.94475012, 1.00516127])}
{'

TypeError: 'NoneType' object is not iterable

In [ ]:
def func(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    return -obj,cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

if re_run_ASBO == True or re_run_save_ASBO == True:
    for i in range(20,runs_number,1):
        optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        try:
            optimizer.maximize(init_points=100, n_iter=200,p_hyper=[[5, 2, 0.78, 0.95,1e-5]],acquisition_function=acquisition_function)
            print(optimizer.max)
            x_1_list.extend([optimizer.max['params']['x1']])
            x_2_list.extend([optimizer.max['params']['x2']])
            x_3_list.extend([optimizer.max['params']['x3']])
            x_4_list.extend([optimizer.max['params']['x4']])
            x_5_list.extend([optimizer.max['params']['x5']])  
            obj_list.extend([optimizer.max['target']])                                  
            con_1_list.extend([optimizer.max['constraint'][0]])
            con_2_list.extend([optimizer.max['constraint'][1]])
            con_3_list.extend([optimizer.max['constraint'][2]])
            con_4_list.extend([optimizer.max['constraint'][3]])
            all_res = [item['target'] for item in optimizer.res]
            all_fea = [item['allowed'] for item in optimizer.res]
            modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
            all_res_list.append(modified_res)            
        except Exception as e:
            print(e)
            obj_list.append(None)
            x_1_list.append(None)
            x_2_list.append(None)
            x_3_list.append(None)
            x_4_list.append(None)
            x_5_list.append(None)                        
            con_1_list.append(None)
            con_2_list.append(None)
            con_3_list.append(None)
            con_4_list.append(None)  
            modified_res = [-np.inf for _ in range(100+200)]
            all_res_list.append(modified_res)   

        df2 = pd.DataFrame(all_res_list, columns=[list(range(100+200))])               
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list
        })
        if re_run_save_ASBO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
            df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)

{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': -1.0850203049721405, 'params': {'x1': 0.005305908607200862, 'x2': -1.9404459172307633, 'x3': 2.719088014660587, 'x4': 1.341367356119223, 'x5': 1.8007126247561698}, 'constraint': array([-0.02197116, -0.4676898 ,  1.22630726,  1.06814573])}
{'target': -1.070656235926411, 'params': {'x1': -0.0003865317442983569, 'x2': -2.95434822653393, 'x3': 4.672399582350155, 'x4': 3.2207009428250726, 'x5': 1.46118937733996}, 'constraint': array([0.78829643, 0.77982467, 1.11843452, 0.74836244])}
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': -5.658661953763168, 'params': {'x1': -1.4043268332545926, 'x2': 17.41916874472771, 'x3': -3.799609426534354, 'x4': 9.756484010120317, 'x5': -0.10899043044127729}, 'constraint': array([ 0.52850341, -0.81711947,  1.148504

In [ ]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 11


,obj,x_1,x_2,x_3,x_4,x_5,con_1,con_2,con_3,con_4
count,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,-1.730761,0.091361,4.115509,-1.268743,5.834035,1.544653,0.572679,0.542894,1.288569,0.735352
std,1.640020,0.735030,6.559536,3.978104,5.425359,4.612191,0.647427,1.101015,0.535235,0.373146
min,-11.177637,-1.404327,-5.243145,-8.488362,-3.479680,-6.797270,-0.851496,-0.977023,-0.061630,0.024539
25%,-1.098503,-0.001619,-1.853324,-4.413724,1.394414,-2.005082,0.122354,-0.347694,1.000687,0.493187
50%,-1.074548,0.000076,2.350463,-1.662630,5.594154,1.580117,0.718406,0.411247,1.143672,0.740845
75%,-1.071549,0.001961,10.073179,2.106270,9.569142,5.084767,1.012713,1.262605,1.632990,1.011801
max,-1.069647,4.955777,17.419169,6.774415,16.784317,10.746632,1.823471,3.374349,2.933291,1.499336


### CBO

In [6]:
re_run_CBO = True
re_run_save_CBO = True
sol = 'CBO'
case_name = 'Case3'
runs_number = 100

In [ ]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []

def func_obj(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    # print(('cons, obj value=', cons, obj))
    return -obj

def func_cons(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    return cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

if re_run_CBO == True or re_run_save_CBO == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        try: 
            optimizer.maximize(init_points=100, n_iter=200,acquisition_function=acquisition_function)
            print(optimizer.max)
            x_1_list.extend([optimizer.max['params']['x1']])
            x_2_list.extend([optimizer.max['params']['x2']])
            x_3_list.extend([optimizer.max['params']['x3']])
            x_4_list.extend([optimizer.max['params']['x4']])
            x_5_list.extend([optimizer.max['params']['x5']])   
            obj_list.extend([optimizer.max['target']])                     
            con_1_list.extend([optimizer.max['constraint'][0]])
            con_2_list.extend([optimizer.max['constraint'][1]])
            con_3_list.extend([optimizer.max['constraint'][2]])
            con_4_list.extend([optimizer.max['constraint'][3]])
        except Exception as e:
            print(e)
            obj_list.append(None)
            x_1_list.append(None)
            x_2_list.append(None)
            x_3_list.append(None)
            x_4_list.append(None)
            x_5_list.append(None)                        
            con_1_list.append(None)
            con_2_list.append(None)
            con_3_list.append(None)
            con_4_list.append(None)          
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,        
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list
        })
        if re_run_save_CBO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not subscriptable
{'target': None, 'params': None, 'constraint': None}
'NoneType' object is not su

In [8]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 96


,obj,x_1,x_2,x_3,x_4,x_5,con_1,con_2,con_3,con_4
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,-4.580155,1.296882,1.867299,-2.550546,5.172214,2.681289,0.500259,-0.318029,0.979080,0.726443
std,3.563612,1.773480,7.525175,5.852490,5.295663,6.448829,1.080816,0.676006,0.765756,0.362304
min,-9.712120,-0.092739,-3.860055,-9.651355,-1.131102,-2.364844,-0.935266,-0.894231,-0.039047,0.437051
25%,-5.145764,0.365803,-2.901239,-5.245259,2.642527,-0.907653,0.041920,-0.760005,0.659216,0.448614
50%,-3.573885,0.695641,-0.674712,-2.503217,5.102894,0.510356,0.669203,-0.498942,1.097704,0.632642
75%,-3.008276,1.626720,4.093826,0.191496,7.632581,4.099298,1.127542,-0.056966,1.417567,0.910471
max,-1.460728,3.888988,12.678676,4.455604,11.614169,12.069289,1.597895,0.619998,1.759958,1.203437


In [ ]:
### Only few feasible solutions. Rerun to save iteration data

In [ ]:
sol = 'CBO'
case_name = 'Case3'
runs_number = 100

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []
all_res_list = []

def func_obj(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    # print(('cons, obj value=', cons, obj))
    return -obj

def func_cons(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    return cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

if re_run_CBO == True or re_run_save_CBO == True:
    for i in [21,22,52,75,95,96,97,98,99]:
        optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        try: 
            optimizer.maximize(init_points=100, n_iter=200,acquisition_function=acquisition_function)
            print(optimizer.max)
            x_1_list.extend([optimizer.max['params']['x1']])
            x_2_list.extend([optimizer.max['params']['x2']])
            x_3_list.extend([optimizer.max['params']['x3']])
            x_4_list.extend([optimizer.max['params']['x4']])
            x_5_list.extend([optimizer.max['params']['x5']])   
            obj_list.extend([optimizer.max['target']])                     
            con_1_list.extend([optimizer.max['constraint'][0]])
            con_2_list.extend([optimizer.max['constraint'][1]])
            con_3_list.extend([optimizer.max['constraint'][2]])
            con_4_list.extend([optimizer.max['constraint'][3]])
            all_res = [item['target'] for item in optimizer.res]
            all_fea = [item['allowed'] for item in optimizer.res]
            modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
            all_res_list.append(modified_res)  

        except Exception as e:
            print(e)
            obj_list.append(None)
            x_1_list.append(None)
            x_2_list.append(None)
            x_3_list.append(None)
            x_4_list.append(None)
            x_5_list.append(None)                        
            con_1_list.append(None)
            con_2_list.append(None)
            con_3_list.append(None)
            con_4_list.append(None)  
            modified_res = [-np.inf for _ in range(100+200)]
            all_res_list.append(modified_res)   
        df2 = pd.DataFrame(all_res_list, columns=[list(range(100+200))])   
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,        
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list
        })
        if re_run_save_CBO == True:
            df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)


{'target': -3.5241246807378244, 'params': {'x1': 0.518650449627383, 'x2': -2.5816333715711806, 'x3': 4.4556035102939076, 'x4': 3.9004039862740965, 'x5': -2.3648443564296318}, 'constraint': array([-0.93526557, -0.71526311,  0.89197088,  1.20343667])}
{'target': -1.4607282595894984, 'params': {'x1': -0.09273923057331213, 'x2': 1.2322087516869862, 'x3': -1.229873091020331, 'x4': -1.1311020676399222, 'x5': 1.4426341645062732}, 'constraint': array([ 0.36764858, -0.89423098,  1.75995843,  0.45246869])}
{'target': -9.712119843725509, 'params': {'x1': 3.8889877318477737, 'x2': -3.8600545386545875, 'x3': -9.65135474412705, 'x4': 11.61416881232957, 'x5': 12.069289039817807}, 'constraint': array([ 0.97075743,  0.61999772, -0.03904692,  0.43705139])}
{'target': -3.623645427296051, 'params': {'x1': 0.8726309888264243, 'x2': 12.678676361557415, 'x3': -3.776560360932466, 'x4': 6.305384987783725, 'x5': -0.4219228934135799}, 'constraint': array([ 1.59789475, -0.28262014,  1.30343686,  0.81281582])}
{'t

In [ ]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

### SCBO

In [5]:
re_run_SCBO = True
re_run_save_SCBO = True
sol = 'SCBO'
case_name = 'Case3'
runs_number = 100

In [6]:
import gpytorch
import torch
import tr_bo
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize
from botorch.models.model_list_gp_regression import ModelListGP

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
iterations = []
max_values = []
all_res_list = []

def func(x):
    obj, c1, c2, c3, c4 = Case3(x)
    return -obj,c1,c2, c3,c4

device = torch.device("cuda" if torch.cuda.is_available() else"cpu")
dtype = torch.double
dim = 5
batch_size = 1
n_init = 100 
bo_iter = 200
max_cholesky_size = float("inf")  # Always use Cholesky

lb = torch.tensor([-1.0, -1, -2.0, 0,], dtype=torch.float64)
ub =  torch.tensor([2.0, 4, 3.0, 1.5, ], dtype=torch.float64)
bounds = torch.tensor([[-5, -20,  -10, -12, -15], [12, 22, 10, 20, 18]])

def objective(x):
    x_numpy = x.numpy()
    res = func(x_numpy)
    # res = func(x)
    target = res[0]
    c1 = res[1]
    c2 = res[2]
    c3 = res[3]
    c4 = res[4]
    return target,c1,c2,c3,c4
def run_multiple_calculations(num_runs):
    for random_seed in range(num_runs):
        try:
            def eval_objective(x):
                return objective(unnormalize(x, bounds))
            # Initialize TuRBO state
            state = tr_bo.ScboState(dim, batch_size=batch_size)
            sobol = SobolEngine(dim, scramble=True, seed=1)

            # Get initial data
            # Must get initial values for both objective and constraints
            train_X = tr_bo.get_initial_points(dim, n_init,seed=random_seed)

            train_Y_tensor_list = []
            C1_tensor_list = []
            C2_tensor_list = []
            C3_tensor_list = []
            C4_tensor_list = []
            for x in train_X:
                result = eval_objective(x)
                combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                train_Y, C1, C2,C3,C4 = combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], combined_tensor[4]
                train_Y_tensor_list.append(train_Y)
                C1_tensor_list.append(C1)
                C2_tensor_list.append(C2)
                C3_tensor_list.append(C3)
                C4_tensor_list.append(C4)
            
            train_Y = torch.stack(train_Y_tensor_list, dim=0)
            C1 = torch.stack(C1_tensor_list, dim=0)
            C2 = torch.stack(C2_tensor_list, dim=0)
            C3 = torch.stack(C3_tensor_list, dim=0)
            C4 = torch.stack(C4_tensor_list, dim=0)
            while not state.restart_triggered:  # Run until TuRBO converges
                # Fit GP models for objective and constraints
                model = tr_bo.get_fitted_model(train_X, train_Y,dim)
                c1_model = tr_bo.get_fitted_model(train_X, C1,dim)
                c2_model = tr_bo.get_fitted_model(train_X, C2,dim)
                c3_model = tr_bo.get_fitted_model(train_X, C3,dim)
                c4_model = tr_bo.get_fitted_model(train_X, C4,dim)
                # Generate a batch of candidates
                with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                    X_next = tr_bo.generate_batch(
                        state=state,
                        model=model,
                        X=train_X,
                        Y=train_Y,
                        batch_size=batch_size,
                        n_candidates=2000,
                        constraint_model=ModelListGP(c1_model, c2_model,c3_model,c4_model),
                        sobol=sobol,
                    )

                Y_next_tensor_list = []
                C1_next_tensor_list = []
                C2_next_tensor_list = []
                C3_next_tensor_list = []
                C4_next_tensor_list = []
                for x in X_next:
                    result = eval_objective(x)
                    combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                    Y_next, C1_next, C2_next, C3_next, C4_next,= combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], combined_tensor[4]
                    Y_next_tensor_list.append(Y_next)
                    C1_next_tensor_list.append(C1_next)
                    C2_next_tensor_list.append(C2_next)
                    C3_next_tensor_list.append(C3_next)
                    C4_next_tensor_list.append(C4_next)
                Y_next = torch.stack(Y_next_tensor_list, dim=0)
                C1_next = torch.stack(C1_next_tensor_list, dim=0)
                C2_next = torch.stack(C2_next_tensor_list, dim=0)
                C3_next = torch.stack(C3_next_tensor_list, dim=0)
                C4_next = torch.stack(C4_next_tensor_list, dim=0)

                C_next = torch.cat([C1_next, C2_next,C3_next, C4_next,], dim=-1)
                state = tr_bo.update_state(state, Y_next, C_next,ub,lb)
                train_X = torch.cat((train_X, X_next), dim=0)
                train_Y = torch.cat((train_Y, Y_next), dim=0)
                C1 = torch.cat((C1, C1_next), dim=0)
                C2 = torch.cat((C2, C2_next), dim=0)
                C3 = torch.cat((C3, C3_next), dim=0)
                C4 = torch.cat((C4, C4_next), dim=0)
                C = torch.cat([C1, C2, C3, C4], dim=-1)
                bool_tensor = (C >= lb.repeat(len(C), 1)) & (C <= ub.repeat(len(C), 1))
                bool_tensor = torch.all(bool_tensor, dim=-1)
                Valid_Y = train_Y[bool_tensor]
                Valid_X = train_X[bool_tensor]

                if len(Valid_Y)!=0:
                    max_index = torch.argmax(Valid_Y)  
                else:
                    print(f"{len(train_X)}) No feasible point yet! TR length: {state.length:.2e}")  

                if len(train_X)==n_init+bo_iter:
                    modified_Y = torch.where(bool_tensor, train_Y.flatten(), torch.tensor(float('-inf')))
                    all_res_list.append(modified_Y.tolist())
                    df2 = pd.DataFrame(all_res_list, columns=[list(range(n_init+bo_iter))])     

                    if len(Valid_Y)!=0:
                        max_value = Valid_Y[max_index].item()
                        c_x = Valid_X[max_index]
                        print("obj= ",max_value, ' x = ', c_x)
                        max_values.append(max_value)
                        x_1_list.append(c_x[0].item())
                        x_2_list.append(c_x[1].item())
                        x_3_list.append(c_x[2].item())
                        x_4_list.append(c_x[3].item())
                        x_5_list.append(c_x[4].item())      
                    else:
                        print("No feasible solutions")
                        max_values.append(None)
                        x_1_list.append(None)
                        x_2_list.append(None)
                        x_3_list.append(None)
                        x_4_list.append(None)
                        x_5_list.append(None)                                                           
                    iterations.append(len(train_X))
                    df = pd.DataFrame({
                    'obj': max_values,
                    'x_1': x_1_list,
                    'x_2': x_2_list,
                    'x_3': x_3_list,
                    'x_4': x_4_list,
                    'x_5': x_5_list,                       
                    'iterations': iterations
                    })
                    if re_run_save_SCBO == True:
                        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
                        df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)
                    break

        except Exception as e:
            fixed_length_Y = torch.full((n_init+bo_iter,), float('-inf'))           
            modified_Y = torch.where(bool_tensor, train_Y.flatten(), torch.tensor(float('-inf')))
            fixed_length_Y[:modified_Y.size(0)] = modified_Y
            all_res_list.append(fixed_length_Y.tolist())
            df2 = pd.DataFrame(all_res_list, columns=[list(range(n_init+bo_iter))])
                        
            if len(train_X) == n_init + bo_iter:
                print(e)
            else:
                print(e, "Early Stop")
                if len(Valid_Y)!=0:
                    max_value = Valid_Y[max_index].item()
                    c_x = Valid_X[max_index]
                    print("obj= ",max_value, ' x = ', c_x)
                    max_values.append(max_value)
                    x_1_list.append(c_x[0].item())
                    x_2_list.append(c_x[1].item())
                    x_3_list.append(c_x[2].item())
                    x_4_list.append(c_x[3].item())
                    x_5_list.append(c_x[4].item())                        
                else:
                    print("obj= ", None, ' x = ', None)
                    max_values.append(None)
                    x_1_list.append(None)
                    x_2_list.append(None)
                    x_3_list.append(None)
                    x_4_list.append(None)
                    x_5_list.append(None)                
            iterations.append(len(train_X))
            df = pd.DataFrame({
            'obj': max_values,
            'x_1': x_1_list,
            'x_2': x_2_list,
            'x_3': x_3_list,
            'x_4': x_4_list,
            'x_5': x_5_list,               
            'iterations': iterations
            })
            if re_run_save_SCBO == True:
                df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
                df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)                

run_multiple_calculations(num_runs=runs_number)

101) No feasible point yet! TR length: 8.00e-01
102) No feasible point yet! TR length: 8.00e-01
103) No feasible point yet! TR length: 8.00e-01
104) No feasible point yet! TR length: 8.00e-01
105) No feasible point yet! TR length: 8.00e-01
106) No feasible point yet! TR length: 8.00e-01
107) No feasible point yet! TR length: 8.00e-01
108) No feasible point yet! TR length: 8.00e-01
109) No feasible point yet! TR length: 8.00e-01
110) No feasible point yet! TR length: 8.00e-01
111) No feasible point yet! TR length: 8.00e-01
112) No feasible point yet! TR length: 8.00e-01
113) No feasible point yet! TR length: 8.00e-01
114) No feasible point yet! TR length: 8.00e-01
115) No feasible point yet! TR length: 6.67e-01
116) No feasible point yet! TR length: 6.67e-01
117) No feasible point yet! TR length: 6.67e-01
118) No feasible point yet! TR length: 6.67e-01
119) No feasible point yet! TR length: 6.67e-01
120) No feasible point yet! TR length: 6.67e-01
121) No feasible point yet! TR length: 5

In [7]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 97


,obj,x_1,x_2,x_3,x_4,x_5,iterations
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,100.00000
mean,-5.990251,0.403784,0.360157,0.766440,0.560869,0.487233,183.64000
std,2.635759,0.079972,0.033173,0.118773,0.085074,0.124756,14.59661
min,-8.031235,0.315703,0.329668,0.658645,0.497440,0.361226,150.00000
25%,-7.478124,0.369755,0.342494,0.702776,0.512531,0.425500,174.75000
50%,-6.925012,0.423807,0.355320,0.746907,0.527622,0.489773,182.50000
75%,-4.969759,0.447824,0.375402,0.820338,0.592584,0.550236,190.00000
max,-3.014507,0.471841,0.395484,0.893768,0.657546,0.610699,227.00000


In [ ]:
**

### trust-constr

In [10]:
re_run_tc = True
re_run_save_tc = True
sol = 'trust-constr'
case_name = 'Case3'
runs_number = 100

In [ ]:
def func(x):
    obj, c1, c2, c3, c4 = Case3(x)
    cons = [c1,c2,c3,c4]
    return obj,c1,c2,c3,c4

initial_x1 = []
initial_x2 = []
initial_x3 = []
initial_x4 = []
initial_x5 = []

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []

nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1,-1,-2,0,])
    nc_ub = np.array([2,  4, 3,1.5])
    cons = NonlinearConstraint(lambda x: func(x)[1:5], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x[0]
    lb = [-5, -20,-10,-12,-15] 
    ub = [12,22,10,20,18] 
    bd = Bounds(lb, ub, True) 
    try:
        result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)
        print('obj = ', result.fun, ', x = ', result.x)
        obj_list.append(result.fun)
        x_1_list.append(result.x[0])
        x_2_list.append(result.x[1])
        x_3_list.append(result.x[2])
        x_4_list.append(result.x[3])
        x_5_list.append(result.x[4])        
        con_1_list.append(result.constr[0][0])
        con_2_list.append(result.constr[0][1])
        con_3_list.append(result.constr[0][2])    
        con_4_list.append(result.constr[0][3])    
        initial_x1.append(x[0][0])  
        initial_x2.append(x[0][1])  
        initial_x3.append(x[0][2])  
        initial_x4.append(x[0][3])  
        initial_x5.append(x[0][4])      
        nfev_r.append(result.nfev)
        success_r.append(str(result.success))
        message_r.append(result.message)

    except Exception as e:
        print(e)
        obj_list.append(None)
        x_1_list.append(None)
        x_2_list.append(None)
        x_3_list.append(None)
        x_4_list.append(None)
        x_5_list.append(None)        
        con_1_list.append(None)
        con_2_list.append(None)
        con_3_list.append(None)    
        con_4_list.append(None)    
        initial_x1.append(x[0][0])  
        initial_x2.append(x[0][1])  
        initial_x3.append(x[0][2])  
        initial_x4.append(x[0][3])  
        initial_x5.append(x[0][4])      
        nfev_r.append(None)
        success_r.append(None)
        message_r.append(e)

    df = pd.DataFrame({ 
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'initial_x3': initial_x3,
        'initial_x4': initial_x4,  
        'initial_x5': initial_x5,                 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save_tc == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    # return result.x

def initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18] ,a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run_tc == True or re_run_save_tc == True:
    x_random = initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)

obj =  16.19991213385418 , x =  [ 9.9786474  -6.24908098  3.43995161  3.23636087 -1.60187169]
obj =  9.669340646269298 , x =  [ 3.94802296 -4.66005439  0.71402314  2.79435424 -4.97649205]
obj =  9.6693406462693 , x =  [ 3.94802296  0.92420133 -4.81294049  7.14812646 -7.81425908]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


obj =  1.0695605577606 , x =  [-5.96153138e-13  1.08914239e+01 -5.43918380e+00  6.47634090e+00
 -4.98441760e+00]
obj =  11.17763579843486 , x =  [ 4.95544977 -5.04929557  2.29742109  4.71933809 -4.78056866]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


obj =  9.669340646269298 , x =  [ 3.94802296 -4.66229285  2.2381043   1.41162261 -2.77795987]
obj =  11.177635798434855 , x =  [ 4.95544977 -8.06440864  4.56198123  3.11391704  3.5531682 ]


...

e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


obj =  9.6693406462693 , x =  [ 3.94802296 -7.52884744 -4.80164798 -1.49467638  7.87093361]
obj =  7.929909251990148 , x =  [-2.93905325 11.28029436  0.33100341 11.98669996 -3.6193476 ]
obj =  14.604818660820976 , x =  [ 7.97147141  9.66952927 -7.80539973  1.01023692  1.37543296]
obj =  12.48575968524666 , x =  [ 5.96168434  1.97633249  2.17155772 16.75904372  6.93277985]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


obj =  12.485759685246657 , x =  [ 5.96168434  2.7801474  -8.05962474 18.01804518 11.33233131]
`x0` violates bound constraints.
obj =  11.177635798434867 , x =  [ 4.95544976  6.66839701 -7.845131    6.60281036  7.34566648]
obj =  7.92990925199015 , x =  [ 2.93905326 -7.49657778  5.16588229  3.08310494  3.81707048]
obj =  12.485759685246657 , x =  [ 5.96168434 -7.76223832  3.63710514  2.33800682  3.70075097]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [11]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df_cleaned = df.dropna(subset=['obj'])
df_cleaned.describe()

NaN count in column 'obj': 14


,obj,x_1,x_2,x_3,x_4,x_5,con_1,con_2,con_3,con_4,initial_x1,initial_x2,initial_x3,initial_x4,initial_x5,nfev
count,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,9.466433,4.040447,0.289832,-0.738800,5.617766,1.178155,0.879651,0.730883,0.532256,0.873133,3.567130,-2.035740,0.262419,4.705077,1.586516,1762.395349
std,4.657100,3.487416,7.246155,4.900574,5.725379,5.179718,0.892429,1.383246,0.691260,0.522628,4.643096,12.173144,5.550714,8.960935,9.018906,2253.877868
min,1.069561,-2.939053,-8.064409,-9.961470,-3.541913,-7.869162,-1.033796,-0.999906,-0.572117,0.032999,-4.839344,-19.895738,-9.739653,-11.518204,-14.889930,156.000000
25%,5.923410,1.167520,-6.036498,-4.841744,1.365967,-2.830949,0.326890,-0.279681,0.019838,0.474342,0.207796,-11.496278,-4.702487,-2.518627,-5.371950,420.000000
50%,9.669341,3.948023,-2.114358,-0.243539,3.862352,0.885726,0.831767,0.506859,0.327717,0.780041,3.901508,-4.341049,0.660995,3.654909,1.501329,717.000000
75%,13.520734,6.774825,6.520037,3.265646,9.880860,4.899223,1.724839,1.541159,0.918522,1.361022,7.099720,8.582313,4.876176,13.127942,8.162247,2214.000000
max,17.400975,11.983972,16.748507,8.463073,18.018045,13.293029,2.210106,3.936457,2.987331,2.234670,11.397368,20.378388,9.777242,19.838408,17.950608,11856.000000


### slsqp

In [12]:
re_run_slsqp = True
re_run_save_slsqp = True
sol = 'slsqp'
case_name = 'Case3'
runs_number = 100

In [ ]:
initial_x1 = []
initial_x2 = []
initial_x3 = []
initial_x4 = []
initial_x5 = []

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []

nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1, c2, c3, c4 = Case3(x)
    cons = [c1,c2,c3,c4]
    return obj,c1,c2,c3,c4

initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1,-1,-2,0,])
    nc_ub = np.array([2,  4, 3,1.5])
    cons = NonlinearConstraint(lambda x: func(x)[1:5], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x[0]
    lb = [-5, -20,-10,-12,-15] 
    ub = [12,22,10,20,18] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='slsqp', bounds=bd,constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)
    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
    x_3_list.append(result.x[2])
    x_4_list.append(result.x[3])
    x_5_list.append(result.x[4])        

    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    initial_x3.append(x[0][2])  
    initial_x4.append(x[0][3])  
    initial_x5.append(x[0][4])      
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    df = pd.DataFrame({ 
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'initial_x3': initial_x3,
        'initial_x4': initial_x4,  
        'initial_x5': initial_x5,                 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save_slsqp == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    return result.x

def initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18] ,a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run_slsqp == True or re_run_save_slsqp == True:
    x_random = initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `kee

obj =  7.929909307435473 , x =  [ 2.93911777 -7.52930733  6.25923009  3.65835472  2.32682189]
obj =  5.923409681485275 , x =  [ 1.92807005 -0.54926051 -3.29186117  3.92623833 -6.8301388 ]
obj =  12.485759920814465 , x =  [ 5.96180955  2.37649039 -6.57369439  4.62502216 -6.98243528]
obj =  9.669340680149482 , x =  [-3.94797393 12.11383009  2.9125091  15.775061    3.49488201]
obj =  14.604818672032152 , x =  [ 7.97144462 -5.20030091  4.13468126  8.57343871 -4.63405   ]
obj =  16.199912145962827 , x =  [ 9.97861984  3.68184093 -9.73374946  2.47867909 -6.98602864]
obj =  14.604818670790783 , x =  [ 7.97144615 -6.58895816  2.08632213  0.65996611  2.35428081]
obj =  11.177635798434869 , x =  [ 4.95544984  7.3489273  -7.4585253   3.51021197 -5.18248605]
obj =  9.669340646269301 , x =  [ 3.94802302 -7.89072543  7.08055092  2.977598    0.04878562]
obj =  3.607999128180549 , x =  [ 0.91377371 -4.42226773  5.07180845  3.45644996  2.16511081]
obj =  1.0695605650138558 , x =  [ 2.56500918e-09 -1.72

e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `fin

obj =  9.669340646324992 , x =  [ -3.94802498   9.23610547 -10.          -3.6753887    5.33036458]
obj =  3.6079993317827967 , x =  [ 0.91366853 -4.42040106  4.43422646  3.42154345  3.35731273]
obj =  5.923409640466765 , x =  [ 1.9279197  10.7290473  -9.80608159 -1.76589215  3.50159912]
obj =  3.6079991230476414 , x =  [ 0.9138409  -1.93840215  7.13211928  9.86399917 -3.40341461]
obj =  5.9234095900467985 , x =  [-1.92798497  3.54827689  0.91092852  8.96282955 -5.49946433]
obj =  16.19991213385409 , x =  [ 9.97864747 -5.63051351  1.57767518  1.06883179  1.49379926]
obj =  17.400974687943666 , x =  [11.9839468   5.66318852 -7.612431    0.17291762 -2.1295196 ]
obj =  1.069560561413049 , x =  [-1.29193115e-09  5.89293855e+00 -4.06502840e+00  2.26576236e-01
 -2.16535728e+00]
obj =  1.0695605592849273 , x =  [-5.39525772e-10  9.98530208e+00 -7.25582778e+00  2.14508702e+00
 -4.64926497e+00]
obj =  1.0695610706179477 , x =  [-1.81322863e-07  4.18716426e-01  8.71389641e+00  1.22976898e+01
  3.

e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `fin

...

e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `fin

obj =  1.0695605630148113 , x =  [-1.85823978e-09  1.38727309e+01 -2.82566753e+00  1.16040533e+01
  3.51420229e+00]
obj =  5.923409747341459 , x =  [-1.92809692  0.53487465 -1.60649354  1.59168425  5.17022256]
obj =  1.0695605681915272 , x =  [-3.68848511e-09 -2.75656911e+00  1.08203560e-02  1.53893295e+01
  9.90220707e+00]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


obj =  1.06956056956145 , x =  [-4.17282631e-09 -1.95003507e+00 -3.21282334e+00  1.94643885e+00
  6.95140828e+00]
obj =  7.929909253525279 , x =  [-2.93906401 14.46689619 -2.67359833 11.44310249 -4.12767374]
obj =  11.177636280544073 , x =  [-4.95563131 18.22279035 -8.00314729 10.08769154 -6.03078704]
obj =  14.604818661740705 , x =  [ 7.97146378  0.88497415  2.13870216 16.96647388  7.0909392 ]
obj =  17.40097467744956 , x =  [11.9839697  11.65229293 -9.66462941 16.53440114  8.49145332]
obj =  1.0695607029039071 , x =  [ 5.13164889e-08  4.61051514e+00 -4.18486083e+00 -2.51557354e+00
  2.09904467e+00]
obj =  7.92990925219398 , x =  [ 2.93904937  8.19563227 -5.74202968  0.94416282 -2.64200601]
obj =  14.604818776101826 , x =  [ 7.97138539 -6.26462775  6.09013004  7.28365374 -2.89915929]
obj =  7.929909254073957 , x =  [ 2.93904077 -6.89303399 -4.92941099 -2.65217625  7.57749594]


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `fin

In [13]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
false_count = (df['success'] == False).sum()
print(f"Count of rows where 'success' is False: {false_count}")
df_filtered = df[df['success'] != False]
df_filtered.describe()

NaN count in column 'obj': 0
Count of rows where 'success' is False: 4


,obj,x_1,x_2,x_3,x_4,x_5,initial_x1,initial_x2,initial_x3,initial_x4,initial_x5,nfev
count,96.000000,9.600000e+01,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.00000
mean,8.270761,2.909297e+00,2.177319,-0.902878,6.000375,0.711606,3.664921,-1.217467,0.417754,3.691780,1.483596,95.18750
std,5.417403,4.468388e+00,7.692135,5.958598,5.647933,4.918666,4.576818,12.483906,5.696051,9.199286,9.140109,50.85093
min,1.069561,-4.955631e+00,-7.912116,-10.000000,-4.243932,-7.341335,-4.839344,-19.895738,-9.739653,-11.850767,-14.889930,33.00000
25%,3.607999,-1.431661e-08,-4.601838,-5.747525,1.132216,-3.368492,0.456337,-11.435264,-4.955332,-3.619093,-6.131302,62.25000
50%,7.929909,1.927945e+00,0.581151,-1.625270,5.516569,0.449810,3.901508,-1.783072,1.006404,2.609188,1.275189,82.50000
75%,12.485760,5.961721e+00,8.455751,4.361051,9.928434,3.476732,7.056913,10.868743,4.964645,11.106830,8.484954,112.50000
max,17.400975,1.198397e+01,21.451930,9.438011,17.648816,12.926851,11.397368,21.180373,9.777242,19.838408,17.950608,308.00000


### PSO

In [9]:
re_run_PSO = True
re_run_save_PSO = True
sol = 'PSO'
case_name = 'Case3'
runs_number = 100

In [10]:
from pyswarm import pso

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []
iterations=[]
state_list = []
itera = 0
def func(x):
    global itera
    obj, c1, c2, c3, c4 = Case3(x)
    cons = [c1+1,2-c1,c2+1,4-c2,c3+2,3-c3,c4,1.5-c4]
    global x_p
    try:
        equal = np.array_equal(x,x_p)
        if not equal:
            itera=itera+1
    except:
        itera=itera+1
    x_p =x
    return obj,cons

if re_run_PSO == True or re_run_save_PSO == True:
    for num in range(runs_number):
        xopt, fopt = pso(lambda x:func(x)[0], lb =[-5, -20,-10,-2,-15] ,ub=[12,22,10,20,18],f_ieqcons=lambda x:func(x)[1:2], swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=200, minstep=1e-8,minfunc=1e-8, debug=False)
        state = np.all(np.array(func(xopt)[1:])>= 0)
        itera=itera-1         
        print( 'fopt =', fopt, 'xopt = ', xopt, 'state = ', state) 
        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(fopt)
        x_1_list.append(xopt[0])
        x_2_list.append(xopt[1])
        x_3_list.append(xopt[2])
        x_4_list.append(xopt[3])
        x_5_list.append(xopt[4])
        state_list.append(state)
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,    
        'iterations': iterations,
        'state':state_list
        })
        if re_run_save_PSO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

Stopping search: Swarm best objective change less than 1e-08
fopt = 3.7006170098477864 xopt =  [-0.61814977 -1.04860156  6.58557086  8.58953073 -1.98267775] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 6.047707767966404 xopt =  [-2.02066626 18.50341216 -5.92585293  8.53396298 -2.03713899] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 5.697717587964146 xopt =  [ 1.41574542 -4.00288989  3.15835897  8.55071857  6.43481709] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 3.195237793871876 xopt =  [ 0.41156997  6.44340306 -1.28898004 10.22614901 -5.62009762] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 10.667066514835613 xopt =  [ 4.23191719  4.34339677 -5.50441094  7.075597   -7.1776129 ] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 16.89008779578243 xopt =  [ 9.62360251 -5.04922961 -0.81117463 -1.25966963  3.08

In [12]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
false_count = (df['obj'] == 1e+100).sum()
print(f"Count of rows where 'obj' is 1e+100: {false_count}")
df_filtered = df[df['obj'] != 1e+100]
df_filtered = df_filtered[df_filtered['state'] == True]
df_filtered.describe()

NaN count in column 'obj': 0
Count of rows where 'obj' is 1e+100: 21


,obj,x_1,x_2,x_3,x_4,x_5,iterations
count,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000
mean,8.301113,3.146601,2.415461,0.320472,8.239534,0.518317,10419.468354
std,5.020168,3.549647,6.555267,3.819952,4.159250,4.955052,4983.573517
min,1.069561,-3.730186,-7.349938,-9.749254,-1.259670,-7.443832,4048.000000
25%,3.655050,0.293559,-2.556153,-1.941907,6.157447,-3.263179,6392.500000
50%,7.929909,2.238068,0.573723,0.149336,8.550719,-0.703403,8728.000000
75%,12.908006,6.047297,8.522466,3.469034,11.213481,4.790890,14703.500000
max,17.584403,12.000000,18.503412,8.093398,17.647226,9.865478,20100.000000


### GA

In [5]:
re_run_GA = True
re_run_save_GA = True
sol = 'GA'
case_name = 'Case3'
runs_number = 100

In [6]:
from sko.GA import GA
from sko.operators import crossover

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []
iterations=[]
state_list = []
itera = 0
def func(x):
    global itera
    obj, c1, c2, c3, c4 = Case3(x)
    cx1 = -c1-1
    cx11=c1-2
    cx2=-1-c2
    cx22=c2-4
    cx3=-2-c3
    cx33=c3-3
    cx4=-c4
    cx44=c4-1.5
    itera=itera+1
    return obj,cx1,cx2,cx3,cx4,cx11,cx22,cx33,cx44

if re_run_GA == True or re_run_save_GA == True:
    for num in range(runs_number):
        ga = GA(func=lambda x:func(x)[0], n_dim=5, size_pop=100, max_iter=200,lb =[-5, -20,-10,-2,-15] ,ub=[12,22,10,20,18], precision=1e-7,prob_mut = 0.001, constraint_ueq=[lambda x:func(x)[i] for i in range(1,9)])
        ga.register(operator_name='crossover', operator=crossover.crossover_2point_prob, crossover_prob=0.5)
        best_x, best_y = ga.run()
        state = np.all(np.array(func(best_x)[1:])<= 0)
        itera=itera-1 
        print( 'obj =', best_y, 'x = ', best_x,'state = ', state)

        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        
        obj_list.append(best_y[0])
        x_1_list.append(best_x[0])
        x_2_list.append(best_x[1])
        x_3_list.append(best_x[2])
        x_4_list.append(best_x[3])
        x_5_list.append(best_x[4])
        state_list.append(state)

        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list, 
        'iterations': iterations,
        'state':state_list
        })
        if re_run_save_GA == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

obj = [1.06956153] x =  [-3.42726708e-07 -1.07288432e+01  6.73738575e+00  1.51013432e+01
  8.65601266e+00] state =  False
obj = [1.06956081] x =  [-8.94069672e-08 -1.38841448e+01  4.32070492e+00  6.49305115e-01
 -3.22588038e-01] state =  False
obj = [1.06956102] x =  [ 1.63912774e-07 -1.17005425e+01  7.96352743e+00  1.22002629e+01
 -3.14349323e+00] state =  False
obj = [1.06956063] x =  [-2.60770321e-08 -4.27204581e+00 -8.40585335e+00  1.77377094e+01
  4.04072548e-01] state =  False
obj = [1.06956063] x =  [-2.60770321e-08 -3.77965796e+00 -5.50047746e+00  1.86821693e+01
  1.02680427e+01] state =  False
obj = [1.06956762] x =  [-2.49594451e-06  4.28504982e-01 -1.24433283e+00  1.13159389e+01
  1.84170122e-01] state =  False
obj = [1.06956063] x =  [-2.60770321e-08 -1.10130916e+01  1.03116509e+00  1.28357089e+01
  5.37464543e+00] state =  False
obj = [1.06956063] x =  [-2.60770321e-08 -6.81095390e+00  1.68830444e+00  9.20258426e+00
 -7.47895219e+00] state =  False
obj = [1.06956063] x =  

In [8]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df_filtered = df[df['state'] == True]
df_filtered.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,x_3,x_4,x_5,iterations
count,1.000000,1.000000e+00,1.00000,1.000000,1.000000,1.000000,1.0
mean,1.069561,-2.607703e-08,13.28378,-1.207459,12.688443,3.509585,180001.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.069561,-2.607703e-08,13.28378,-1.207459,12.688443,3.509585,180001.0
25%,1.069561,-2.607703e-08,13.28378,-1.207459,12.688443,3.509585,180001.0
50%,1.069561,-2.607703e-08,13.28378,-1.207459,12.688443,3.509585,180001.0
75%,1.069561,-2.607703e-08,13.28378,-1.207459,12.688443,3.509585,180001.0
max,1.069561,-2.607703e-08,13.28378,-1.207459,12.688443,3.509585,180001.0


In [ ]:
(180001-1)/9=20000